<a href="https://colab.research.google.com/github/cchang-vassar/Semantic-Relations-in-Vector-Embeddings/blob/main/study3_3_%5Bnomic%5Dautoencoder_choose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [nomic-002] Autoencoder: Choose Corresponding Embedding

Given an embedding, can a model be trained to choose the correct embeddings corresponding to its counterargument from a list of them?

## Set Up

### Imports

In [ ]:
# General imports
import os
import subprocess
import zipfile
import shutil
import time
from google.colab import userdata
import pickle
import statistics
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
from scipy import spatial
from tenacity import (
  retry,
  stop_after_attempt,
  wait_random_exponential
)

### OSF Setup

In [ ]:
!pip install osfclient

In [ ]:
os.environ["OSF_USERNAME"] = userdata.get("OSF_USERNAME")
OSF_USERNAME = os.environ["OSF_USERNAME"]

In [ ]:
os.environ["OSF_PASSWORD"] = userdata.get("OSF_PASSWORD")
OSF_PASSWORD = os.environ["OSF_PASSWORD"]

In [ ]:
os.environ["OSF_TOKEN"] = userdata.get("OSF_TOKEN")
OSF_TOKEN = os.environ["OSF_TOKEN"]

In [ ]:
os.environ["OSF_PROJECT_ID"] = userdata.get("OSF_PROJECT_ID")
OSF_PROJECT_ID = os.environ["OSF_PROJECT_ID"]

### Nomic Setup

In [ ]:
!pip install nomic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.1 MB/s eta 0:00:00
  Created wheel for nomic: filename=nomic-3.0.27-py3-none-any.whl size=44143 sha256=af755e73ef061b1917e620e93fccb6caf82e71dde96c9c619da5262233b4dfa5
  Stored in directory: /root/.cache/pip/wheels/ab/25/ff/c1bed9c1ce0c6e40846a45cec6b3826793f3d89006026ad251
Successfully built nomic


In [ ]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 5.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.8 MB/s eta 0:00:00


In [ ]:
import nomic
from sentence_transformers import SentenceTransformer
from huggingface_hub import login
login(token=userdata.get('HF_TOKEN'))

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Load Corpora Data

### GPR

In [ ]:
subprocess.run("osf -p sakjg fetch --force osfstorage/corpora/gpr_corpus.zip", shell=True)
print("gpr_corpus.zip successfully imported")
gpr_corpus_file_path_zip = 'gpr_corpus.zip'
gpr_corpus_file_path = 'corpora/gpr-corpus'
with zipfile.ZipFile(gpr_corpus_file_path_zip, 'r') as zip_ref:
  zip_ref.extractall(gpr_corpus_file_path)
extracted_files = os.listdir(gpr_corpus_file_path)
print("Files extracted:", extracted_files)

gpr_corpus.zip successfully imported
Files extracted: ['gpr_corpus', '__MACOSX']


### EACL

In [ ]:
subprocess.run("osf -p sakjg fetch --force osfstorage/corpora/eacl_corpus.zip", shell=True)
print("eacl_corpus.zip successfully imported")
eacl_corpus_file_path_zip = 'eacl_corpus.zip'
eacl_corpus_file_path = 'corpora/eacl-corpus'
with zipfile.ZipFile(eacl_corpus_file_path_zip, 'r') as zip_ref:
  zip_ref.extractall(eacl_corpus_file_path)
extracted_files = os.listdir(eacl_corpus_file_path)
print("Files extracted:", extracted_files)

eacl_corpus.zip successfully imported
Files extracted: ['__MACOSX', 'eacl_corpus']


### persuade_corpus

In [ ]:
subprocess.run("osf -p sakjg fetch --force osfstorage/corpora/persuade_corpus.zip", shell=True)
print("persuade_corpus.zip successfully imported")
persuade_corpus_file_path_zip = 'persuade_corpus.zip'
persuade_corpus_file_path = 'corpora/persuade-corpus'
with zipfile.ZipFile(persuade_corpus_file_path_zip, 'r') as zip_ref:
  zip_ref.extractall(persuade_corpus_file_path)
extracted_files = os.listdir(persuade_corpus_file_path)
print("Files extracted:", extracted_files)

persuade_corpus.zip successfully imported
Files extracted: ['persuade_corpus', '__MACOSX']


### SciFact

In [ ]:
subprocess.run("osf -p sakjg fetch --force osfstorage/corpora/scifact_corpus.zip", shell=True)
print("scifact_corpus.zip successfully imported")
scifact_corpus_file_path_zip = 'scifact_corpus.zip'
scifact_corpus_file_path = 'corpora/scifact-corpus'
with zipfile.ZipFile(scifact_corpus_file_path_zip, 'r') as zip_ref:
  zip_ref.extractall(scifact_corpus_file_path)
extracted_files = os.listdir(scifact_corpus_file_path)
print("Files extracted:", extracted_files)

scifact_corpus.zip successfully imported
Files extracted: ['scifact_corpus', '__MACOSX']


## Data

### [Imports] Get Embeddings

In [ ]:
DIM_EMBEDDING = 768
model = SentenceTransformer("nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True)

modules.json:   0%|          | 0.00/255 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/140 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/69.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

### GPR 55

In [ ]:
gpr_df = pd.read_csv("corpora/gpr-corpus/gpr_corpus/GPR-KB-55/GPR-KB-55.csv")

In [ ]:
def gpr_get_embeddings_df(gpr_df: pd.DataFrame) -> pd.DataFrame:
  gpr_embeddings_df = pd.DataFrame()
  arguments_list = list(gpr_df)

  claims_embeddings = model.encode(['search_document: ' + argument for argument in arguments_list])
  claims_embeddings_df = pd.DataFrame(claims_embeddings, columns=[str(i) for i in range(DIM_EMBEDDING)])
  claims_embeddings_df = pd.concat([gpr_df, claims_embeddings_df], axis=1)
  claims_embeddings_df = claims_embeddings_df.reset_index(drop=True)

  return claims_embeddings_df

In [ ]:
gpr_claims_df = gpr_get_embeddings_df(gpr_df['claim'])
gpr_rebuttals_df = gpr_get_embeddings_df(gpr_df['rebuttal'])
gpr_x_test = gpr_claims_df.select_dtypes(include=[np.number])
gpr_y_test = gpr_rebuttals_df.select_dtypes(include=[np.number])
gpr_combined = pd.concat([gpr_claims_df, gpr_rebuttals_df])
gpr_combined = gpr_combined.reset_index(drop=True)
gpr_combined_nums = gpr_combined.select_dtypes(include=[np.number])

In [ ]:
def metric_choose_argument_gpr(y_true, y_pred):
  """See if the output vector is closest to the rebuttal to the claim"""
  gpr_training_df_32 = tf.cast(gpr_combined_nums, dtype=tf.float32)
  gpr_norm = tf.norm(gpr_training_df_32, axis=1)

  cos_sim_pred = tf.matmul(gpr_training_df_32, y_pred, transpose_b=True) / tf.reshape(tf.norm(y_pred) * gpr_norm, [-1, 1])
  cos_sim_true = tf.matmul(gpr_training_df_32, y_true, transpose_b=True) / tf.reshape(tf.norm(y_true) * gpr_norm, [-1, 1])

  max_cos_sim_pred = tf.math.argmax(cos_sim_pred)
  max_cos_sim_true = tf.math.argmax(cos_sim_true)

  return tf.math.count_nonzero(tf.equal(max_cos_sim_pred, max_cos_sim_true))

### EACL

In [ ]:
eacl_df = pd.read_csv("corpora/eacl-corpus/eacl_corpus/claim_stance_dataset.csv")
eacl_df = eacl_df[['topicId', 'topicText', 'claims.stance', 'claims.claimCorrectedText']]

In [ ]:
topic_lens = []
pro_lens = []
con_lens = []
for topic in eacl_df['topicId'].unique():
  topic_rows = eacl_df[eacl_df['topicId'] == topic]
  topic_lens.append(len(topic_rows))
  pro_lens.append(len(topic_rows[topic_rows['claims.stance'] == "PRO"]))
  con_lens.append(len(topic_rows[topic_rows['claims.stance'] == "CON"]))

In [ ]:
def eacl_get_embeddings_df(eacl_df: pd.DataFrame) -> pd.DataFrame:
  eacl_embeddings_df = pd.DataFrame()
  arguments_list = list(eacl_df['claims.claimCorrectedText'])

  embeddings = model.encode(['search_document: ' + argument for argument in arguments_list])
  embeddings_df = pd.DataFrame(embeddings, columns=[str(i) for i in range(DIM_EMBEDDING)])
  embeddings_df = pd.concat([eacl_df, embeddings_df], axis=1)
  embeddings_df = embeddings_df.reset_index(drop=True)

  return embeddings_df

In [ ]:
eacl_embeddings_df = eacl_get_embeddings_df(eacl_df)

In [ ]:
eacl_nums_df = eacl_embeddings_df.select_dtypes(include=[np.number])
eacl_vectors_df = eacl_nums_df.drop('topicId', axis=1)

### Persuade Corpus

In [ ]:
persuade_corpus = pd.read_csv('/content/corpora/persuade-corpus/persuade_corpus/persuade_corpus_1.0.csv')
persuade_source = pd.read_csv('/content/corpora/persuade-corpus/persuade_corpus/persuade_2.0_human_scores_demo_id_github.csv')

In [ ]:
persuade_corpus = persuade_corpus[persuade_corpus['discourse_type'] == "Evidence"]
persuade_corpus = persuade_corpus.drop(columns=['competition_set', 'full_text', 'discourse_id', 'discourse_start', 'discourse_end', 'discourse_type_num'])

In [ ]:
persuade_source = persuade_source.drop(columns=['full_text', 'holistic_essay_score', 'word_count', 'task', 'assignment', 'source_text', 'gender', 'grade_level', 'ell_status', 'race_ethnicity', 'economically_disadvantaged', 'student_disability_status'])

In [ ]:
persuade_source_dict = zip(persuade_source['essay_id_comp'], persuade_source['prompt_name'])

In [ ]:
persuade_source_dict = dict(persuade_source_dict)

In [ ]:
persuade_corpus['argument'] = [persuade_source_dict[essay_id] if essay_id in persuade_source_dict.keys() else "" for essay_id in persuade_corpus['essay_id_comp']]

In [ ]:
persuade_corpus = persuade_corpus[persuade_corpus['argument'] != ""]

In [ ]:
persuade_corpus = persuade_corpus.drop(columns=['discourse_type', 'essay_id_comp'])
persuade_corpus = persuade_corpus.rename(columns={'discourse_text': 'evidence'})

In [ ]:
persuade_corpus = persuade_corpus.reset_index(drop=True)

In [ ]:
persuade_argument_dict = {
    'Phones and driving': 'Drivers should not be allowed to use phones while driving',
    'Car-free cities': 'We should develop cities to be car-free from now on',
    'Summer projects': 'Summer projects are valuable learning opportunities for students',
    '"A Cowboy Who Rode the Waves"': "The Seagoing Cowboys' work is adventurous, meaningful and transformative",
    'Mandatory extracurricular activities': "Extracurricular activities play an irreplacable role in students' education",
    'Exploring Venus': 'Venus is a challenging but rewarding planet to explore',
    'Facial action coding system': 'Having a large-scale software that analyzes and codifies human facial expressions is pointless',
    'The Face on Mars': 'The face on Mars suggests alien activity in the universe',
    'Community service': 'Doing community service is important to both societal and personal benefit',
    'Grades for extracurricular activities': 'Extracurricular activities should not be graded',
    'Driverless cars': 'Driverless cars are the future and should be fully embraced',
    'Does the electoral college work?': 'The electoral college does not work',
    'Cell phones at school': 'Students should be allowed to bring cell phones to school',
    'Distance learning': 'Online classes are the bane of real education',
    'Seeking multiple opinions': 'It is always a good idea to seek the opinions of multiple people'
}

In [ ]:
persuade_corpus['argument'] = [persuade_argument_dict[topic] for topic in persuade_corpus['argument']]

In [ ]:
persuade_arguments_df = pd.DataFrame(persuade_argument_dict.values(), columns=['argument'])

In [ ]:
def persuade_get_embeddings_df(persuade_df: pd.DataFrame, column_name: str) -> pd.DataFrame:
  persuade_embeddings_df = pd.DataFrame()
  arguments_list = list(persuade_df[column_name])

  embeddings = model.encode(['search_document: ' + argument for argument in arguments_list])
  embeddings_df = pd.DataFrame(embeddings, columns=[str(i) for i in range(DIM_EMBEDDING)])
  embeddings_df = pd.concat([persuade_df, embeddings_df], axis=1)
  embeddings_df = embeddings_df.reset_index(drop=True)

  return embeddings_df

In [ ]:
persuade_arguments_embeddings_df = persuade_get_embeddings_df(persuade_arguments_df, 'argument')

In [ ]:
persuade_arguments_vector_df = persuade_arguments_embeddings_df.select_dtypes(include=[np.number])

In [ ]:
persuade_evidence_embeddings_df = persuade_get_embeddings_df(persuade_corpus, 'evidence')

In [ ]:
persuade_evidence_vector_df = persuade_evidence_embeddings_df.select_dtypes(include=[np.number])

### SciFact Corpus

In [ ]:
DIM_EMBEDDING = 768

def scifact_get_embeddings_df(scifact_df: pd.DataFrame, column_name: str) -> pd.DataFrame:
  """Add embeddings column to a df"""
  embeddings_df = pd.DataFrame()
  arguments_list = list(scifact_df[column_name])

  embeddings = model.encode(['search_document: ' + argument for argument in arguments_list])
  embeddings_df = pd.DataFrame(embeddings, columns=[str(i) for i in range(DIM_EMBEDDING)])
  embeddings_df = pd.concat([scifact_df, embeddings_df], axis=1)
  return embeddings_df

In [ ]:
scifact_corpus = pd.read_json('/content/corpora/scifact-corpus/scifact_corpus/corpus.jsonl', lines=True)

In [ ]:
scifact_evidence_corpus = []

for abstract in scifact_corpus['abstract']:
  for sentence in abstract:
    scifact_evidence_corpus.append(sentence)

In [ ]:
scifact_evidence_embeddings_corpus = scifact_get_embeddings_df(pd.DataFrame(scifact_evidence_corpus, columns=['evidence']), 'evidence')

In [ ]:
scifact_evidence_embeddings = scifact_evidence_embeddings_corpus.select_dtypes(include=[np.number])

In [ ]:
scifact_test = pd.read_json('/content/corpora/scifact-corpus/scifact_corpus/claims_test.jsonl', lines=True)

In [ ]:
scifact_test = scifact_test.drop(columns=['id'])

In [ ]:
scifact_test_embeddings_corpus = scifact_get_embeddings_df(scifact_test, 'claim')

In [ ]:
scifact_test_embeddings = scifact_test_embeddings_corpus.select_dtypes(include=[np.number])

## Autoencoder Model

### Counterargument model

In [ ]:
subprocess.run("osf -p sakjg fetch --force osfstorage/data-dump/nomic-autoencoder/nomic_autoencoder.zip", shell=True)
print("nomic_autoencoder.zip successfully imported")
nomic_autoencoder_file_path_zip = 'nomic_autoencoder.zip'
nomic_autoencoder_file_path = 'current-data-dump/nomic-autoencoder'
with zipfile.ZipFile(nomic_autoencoder_file_path_zip, 'r') as zip_ref:
  zip_ref.extractall(nomic_autoencoder_file_path)
extracted_files = os.listdir(nomic_autoencoder_file_path)
print("Files extracted:", extracted_files)

nomic_autoencoder.zip successfully imported
Files extracted: ['nomic_shuffled_sd_autoencoder_weights.keras', 'nomic_cla_autoencoder_weights.keras', 'nomic_clu_autoencoder_model.keras', 'x_test_cla.pkl', 'nomic_sd_training_plot.png', 'nomic_shuffled_sq_training_log.csv', 'y_test_sd.pkl', 'nomic_cla_training_log.csv', 'nomic_shuffled_cla_autoencoder_model.keras', 'x_train_cla.pkl', 'y_test_cla.pkl', 'nomic_shuffled_sq_autoencoder_model.keras', 'y_train_sq.pkl', 'x_test_sq.pkl', 'nomic_shuffled_clu_autoencoder_model.keras', 'nomic_shuffled_cla_training_log.csv', 'x_train_clu.pkl', 'nomic_clu_training_plot.png', 'nomic_sd_autoencoder_weights.keras', 'y_train_clu.pkl', 'nomic_cla_training_plot.png', 'y_train_cla.pkl', 'x_test_sd.pkl', 'nomic_clu_autoencoder_weights.keras', 'nomic_shuffled_sd_autoencoder_model.keras', 'nomic_shuffled_sd_training_log.csv', 'y_test_sq.pkl', 'nomic_sd_training_log.csv', 'nomic_sq_training_log.csv', 'nomic_shuffled_clu_autoencoder_weights.keras', 'nomic_shuffled

In [ ]:
x_train = pd.read_pickle('current-data-dump/nomic-autoencoder/x_train_sd.pkl')
y_train = pd.read_pickle('current-data-dump/nomic-autoencoder/y_train_sd.pkl')
x_test = pd.read_pickle('current-data-dump/nomic-autoencoder/x_test_sd.pkl')
y_test = pd.read_pickle('current-data-dump/nomic-autoencoder/y_test_sd.pkl')

### Evidence model

In [ ]:
subprocess.run("osf -p sakjg fetch --force osfstorage/data-dump/nomic-evidence-autoencoder/nomic_evidence_autoencoder.zip", shell=True)
print("nomic_evidence_autoencoder.zip successfully imported")
nomic_autoencoder_file_path_zip = 'nomic_evidence_autoencoder.zip'
nomic_autoencoder_file_path = 'current-data-dump/nomic-evidence-autoencoder'
with zipfile.ZipFile(nomic_autoencoder_file_path_zip, 'r') as zip_ref:
  zip_ref.extractall(nomic_autoencoder_file_path)
extracted_files = os.listdir(nomic_autoencoder_file_path)
print("Files extracted:", extracted_files)

Files extracted: ['global_x_test.pkl', 'global_shuffled_training_log.csv', 'global_shuffled_autoencoder_model.keras', 'global_y_test.pkl', 'global_y_train.pkl', 'global_autoencoder_model.keras', 'global_x_train.pkl', 'global_training_log.csv']


In [ ]:
x_evidence_train = pd.read_pickle('current-data-dump/nomic-evidence-autoencoder/global_x_train.pkl')
y_evidence_train = pd.read_pickle('current-data-dump/nomic-evidence-autoencoder/global_y_train.pkl')
x_evidence_test = pd.read_pickle('current-data-dump/nomic-evidence-autoencoder/global_x_test.pkl')
y_evidence_test = pd.read_pickle('current-data-dump/nomic-evidence-autoencoder/global_y_test.pkl')

### SciFact models

In [ ]:
subprocess.run("osf -p sakjg fetch --force osfstorage/data-dump/nomic-scifact-autoencoder/nomic_scifact_autoencoder.zip", shell=True)
print("nomic_scifact_autoencoder.zip successfully imported")
nomic_autoencoder_file_path_zip = 'nomic_scifact_autoencoder.zip'
nomic_autoencoder_file_path = 'current-data-dump/nomic-scifact-autoencoder'
with zipfile.ZipFile(nomic_autoencoder_file_path_zip, 'r') as zip_ref:
  zip_ref.extractall(nomic_autoencoder_file_path)
extracted_files = os.listdir(nomic_autoencoder_file_path)
print("Files extracted:", extracted_files)

nomic_scifact_autoencoder.zip successfully imported
Files extracted: ['combined_training_plot.png', 'counter_autoencoder_model.keras', 'pro_shuffled_autoencoder_model.keras', 'combined_autoencoder_model.keras', 'all_combined_training_plot.png', 'x_train_combined.pkl', 'y_test_counter.pkl', 'counter_training_log.csv', 'combined_shuffled_autoencoder_model.keras', 'combined_shuffled_training_plot.png', 'counter_training_plot.png', 'combined_training_log.csv', 'pro_shuffled_training_log.csv', 'y_test_pro.pkl', 'y_train_counter.pkl', 'y_train_pro.pkl', 'y_train_combined.pkl', 'combined_shuffled_training_log.csv', 'all_pro_training_plot.png', 'x_train_counter.pkl', 'pro_training_plot.png', 'x_test_counter.pkl', 'all_counter_training_plot.png', 'counter_shuffled_training_plot.png', 'counter_shuffled_training_log.csv', 'counter_shuffled_autoencoder_model.keras', 'pro_autoencoder_model.keras', 'x_test_pro.pkl', 'pro_shuffled_training_plot.png', 'y_test_combined.pkl', 'x_train_pro.pkl', 'pro_tra

In [ ]:
x_scifact_train = pd.read_pickle('current-data-dump/nomic-scifact-autoencoder/x_train_combined.pkl')
y_scifact_train = pd.read_pickle('current-data-dump/nomic-scifact-autoencoder/y_train_combined.pkl')
x_scifact_test = pd.read_pickle('current-data-dump/nomic-scifact-autoencoder/x_test_combined.pkl')
y_scifact_test = pd.read_pickle('current-data-dump/nomic-scifact-autoencoder/y_test_combined.pkl')

In [ ]:
x_scifact_pro_train = pd.read_pickle('current-data-dump/nomic-scifact-autoencoder/x_train_pro.pkl')
y_scifact_pro_train = pd.read_pickle('current-data-dump/nomic-scifact-autoencoder/y_train_pro.pkl')
x_scifact_pro_test = pd.read_pickle('current-data-dump/nomic-scifact-autoencoder/x_test_pro.pkl')
y_scifact_pro_test = pd.read_pickle('current-data-dump/nomic-scifact-autoencoder/y_test_pro.pkl')

In [ ]:
x_scifact_counter_train = pd.read_pickle('current-data-dump/nomic-scifact-autoencoder/x_train_counter.pkl')
y_scifact_counter_train = pd.read_pickle('current-data-dump/nomic-scifact-autoencoder/y_train_counter.pkl')
x_scifact_counter_test = pd.read_pickle('current-data-dump/nomic-scifact-autoencoder/x_test_counter.pkl')
y_scifact_counter_test = pd.read_pickle('current-data-dump/nomic-scifact-autoencoder/y_test_counter.pkl')

### Metric

In [ ]:
@tf.keras.saving.register_keras_serializable()
def metric_choose_argument_global_sd_y_train(y_true, y_pred):
  global_training_df_32 = tf.cast(pd.concat([x_evidence_train, y_evidence_train, x_evidence_test, y_evidence_test]), dtype=tf.float32)

  cos_sim_pred = tf.matmul(global_training_df_32, y_pred, transpose_b=True) / tf.reshape(tf.norm(y_pred) * tf.norm(global_training_df_32, axis=1), [-1, 1])
  cos_sim_true = tf.matmul(global_training_df_32, y_true, transpose_b=True) / tf.reshape(tf.norm(y_true) * tf.norm(global_training_df_32, axis=1), [-1, 1])

  max_cos_sim_pred = tf.math.argmax(cos_sim_pred)
  max_cos_sim_true = tf.math.argmax(cos_sim_true)

  return tf.math.count_nonzero(tf.equal(max_cos_sim_pred, max_cos_sim_true))

In [ ]:
@tf.keras.saving.register_keras_serializable()
def metric_choose_argument_combined_y_train(y_true, y_pred):
  global_training_df_32 = tf.cast(pd.concat([x_scifact_train, y_scifact_train, x_scifact_test, y_scifact_test]), dtype=tf.float32)

  cos_sim_pred = tf.matmul(global_training_df_32, y_pred, transpose_b=True) / tf.reshape(tf.norm(y_pred) * tf.norm(global_training_df_32, axis=1), [-1, 1])
  cos_sim_true = tf.matmul(global_training_df_32, y_true, transpose_b=True) / tf.reshape(tf.norm(y_true) * tf.norm(global_training_df_32, axis=1), [-1, 1])

  max_cos_sim_pred = tf.math.argmax(cos_sim_pred)
  max_cos_sim_true = tf.math.argmax(cos_sim_true)

  return tf.math.count_nonzero(tf.equal(max_cos_sim_pred, max_cos_sim_true))

In [ ]:
@tf.keras.saving.register_keras_serializable()
def metric_choose_argument_pro_y_train(y_true, y_pred):
  global_training_df_32 = tf.cast(pd.concat([x_scifact_pro_train, y_scifact_pro_train, x_scifact_pro_test, y_scifact_pro_test]), dtype=tf.float32)

  cos_sim_pred = tf.matmul(global_training_df_32, y_pred, transpose_b=True) / tf.reshape(tf.norm(y_pred) * tf.norm(global_training_df_32, axis=1), [-1, 1])
  cos_sim_true = tf.matmul(global_training_df_32, y_true, transpose_b=True) / tf.reshape(tf.norm(y_true) * tf.norm(global_training_df_32, axis=1), [-1, 1])

  max_cos_sim_pred = tf.math.argmax(cos_sim_pred)
  max_cos_sim_true = tf.math.argmax(cos_sim_true)

  return tf.math.count_nonzero(tf.equal(max_cos_sim_pred, max_cos_sim_true))

In [ ]:
@tf.keras.saving.register_keras_serializable()
def metric_choose_argument_counter_y_train(y_true, y_pred):
  global_training_df_32 = tf.cast(pd.concat([x_scifact_counter_train, y_scifact_counter_train, x_scifact_counter_test, y_scifact_counter_test]), dtype=tf.float32)

  cos_sim_pred = tf.matmul(global_training_df_32, y_pred, transpose_b=True) / tf.reshape(tf.norm(y_pred) * tf.norm(global_training_df_32, axis=1), [-1, 1])
  cos_sim_true = tf.matmul(global_training_df_32, y_true, transpose_b=True) / tf.reshape(tf.norm(y_true) * tf.norm(global_training_df_32, axis=1), [-1, 1])

  max_cos_sim_pred = tf.math.argmax(cos_sim_pred)
  max_cos_sim_true = tf.math.argmax(cos_sim_true)

  return tf.math.count_nonzero(tf.equal(max_cos_sim_pred, max_cos_sim_true))

## Load saved model

In [ ]:
global_autoencoder_model = tf.keras.models.load_model('current-data-dump/nomic-autoencoder/nomic_sd_autoencoder_model.keras')

In [ ]:
global_evidence_autoencoder_model = tf.keras.models.load_model('current-data-dump/nomic-evidence-autoencoder/global_autoencoder_model.keras')

In [ ]:
global_scifact_autoencoder_model = tf.keras.models.load_model('/content/current-data-dump/nomic-scifact-autoencoder/combined_autoencoder_model.keras')

In [ ]:
global_scifact_pro_autoencoder_model = tf.keras.models.load_model('/content/current-data-dump/nomic-scifact-autoencoder/pro_autoencoder_model.keras')

In [ ]:
global_scifact_counter_autoencoder_model = tf.keras.models.load_model('/content/current-data-dump/nomic-scifact-autoencoder/counter_autoencoder_model.keras')

## GPR predict (Counterargument)

In [ ]:
global_autoencoder_gpr_predictions = global_autoencoder_model.predict(gpr_x_test)
global_autoencoder_gpr_predictions_df = pd.DataFrame(global_autoencoder_gpr_predictions)
global_autoencoder_gpr_predictions_df.columns = [str(i) for i in global_autoencoder_gpr_predictions_df.columns]

2/2 [==============================] - 1s 6ms/step


In [ ]:
successes = 0
for i in range(len(gpr_y_test)):
  gpr_y_test_tf = tf.convert_to_tensor(gpr_y_test.loc[i], dtype=tf.float32)
  gpr_pred_tf = tf.convert_to_tensor(global_autoencoder_gpr_predictions_df.loc[i], dtype=tf.float32)
  gpr_y_test_tf = tf.reshape(gpr_y_test_tf, (1, -1))
  gpr_pred_tf = tf.reshape(gpr_pred_tf, (1, -1))
  if metric_choose_argument_gpr(gpr_y_test_tf, gpr_pred_tf).numpy() == 1:
    successes += 1

In [ ]:
gpr_success_rate = successes / len(gpr_y_test) * 100

In [ ]:
gpr_success_rate

14.545454545454545

## EACL Predict (Counterargument)

In [ ]:
global_autoencoder_eacl_predictions = global_evidence_autoencoder_model.predict(eacl_vectors_df)
global_autoencoder_eacl_predictions_df = pd.DataFrame(global_autoencoder_eacl_predictions)
global_autoencoder_eacl_predictions_df.columns = [str(i) for i in global_autoencoder_eacl_predictions_df.columns]

75/75 [==============================] - 0s 2ms/step


In [ ]:
eacl_topk = 500
pred_topk = []

In [ ]:
eacl_embeddings_df_32 = tf.cast(eacl_vectors_df, dtype=tf.float32)
global_autoencoder_eacl_predictions_tf = tf.constant(global_autoencoder_eacl_predictions_df.values, dtype=tf.float32)
eacl_embeddings_norm = tf.norm(eacl_embeddings_df_32, axis=1)
eacl_topics = list(eacl_embeddings_df['topicId'])
eacl_stances = list(eacl_embeddings_df['claims.stance'])

In [ ]:
for i, row in enumerate(global_autoencoder_eacl_predictions_tf):
  successes = 0
  y_pred = tf.reshape(row, [1, -1])
  target_topic = eacl_topics[i]
  target_type = 'PRO' if eacl_stances[i] == 'CON' else 'CON'

  cos_sim_pred = tf.matmul(eacl_embeddings_df_32, y_pred, transpose_b=True) / tf.reshape((tf.norm(y_pred) * eacl_embeddings_norm), [-1, 1])
  top_k_sim_pred = tf.math.top_k(tf.reshape(cos_sim_pred, [-1]), k=eacl_topk).indices.numpy()

  for index in top_k_sim_pred:
    if eacl_topics[index] == target_topic and eacl_stances[index] == target_type:
      successes += 1
  pred_topk.append(successes / eacl_topk * 100)

In [ ]:
eacl_topk_success_rate = statistics.mean(pred_topk)

In [ ]:
eacl_topk_success_rate

6.818295739348371

## persuade Predict (Evidence)

In [ ]:
global_evidence_autoencoder_persuade_predictions = global_evidence_autoencoder_model.predict(persuade_arguments_vector_df)
global_evidence_autoencoder_persuade_predictions_df = pd.DataFrame(global_evidence_autoencoder_persuade_predictions)
global_evidence_autoencoder_persuade_predictions_df.columns = [str(i) for i in global_evidence_autoencoder_persuade_predictions_df.columns]

1/1 [==============================] - 0s 32ms/step


In [ ]:
persuade_topk = 10000
pred_topk=[]

In [ ]:
persuade_embeddings_df_32 = tf.cast(persuade_evidence_vector_df, dtype=tf.float32)
global_autoencoder_persuade_predictions_tf = tf.constant(global_evidence_autoencoder_persuade_predictions_df.values, dtype=tf.float32)
persuade_embeddings_norm = tf.norm(persuade_embeddings_df_32, axis=1)
persuade_topics = list(persuade_evidence_embeddings_df['argument'])

In [ ]:
for i, row in enumerate(global_autoencoder_persuade_predictions_tf):
  successes = 0
  y_pred = tf.reshape(row, [1, -1])
  target_topic = persuade_topics[i]

  cos_sim_pred = tf.matmul(persuade_embeddings_df_32, y_pred, transpose_b=True) / tf.reshape((tf.norm(y_pred) * persuade_embeddings_norm), [-1, 1])
  top_k_sim_pred = tf.math.top_k(tf.reshape(cos_sim_pred, [-1]), k=persuade_topk).indices.numpy()

  for index in top_k_sim_pred:
    if persuade_topics[index] == target_topic:
      successes += 1
  pred_topk.append(successes / persuade_topk * 100)

In [ ]:
persuade_topk_success_rate = statistics.mean(pred_topk)

In [ ]:
persuade_topk_success_rate

7.71

## Scifact Predict (Evidence + Combined + Qualitative)

In [ ]:
global_autoencoder_scifact_predictions = global_scifact_autoencoder_model.predict(scifact_test_embeddings)
global_autoencoder_scifact_predictions_df = pd.DataFrame(global_autoencoder_scifact_predictions)
global_autoencoder_scifact_predictions_df.columns = [str(i) for i in global_autoencoder_scifact_predictions_df.columns]

10/10 [==============================] - 1s 2ms/step


In [ ]:
scifact_topk = 10
pred_topk=[]

In [ ]:
scifact_embeddings_df_32 = tf.cast(scifact_evidence_embeddings, dtype=tf.float32)
global_autoencoder_scifact_predictions_tf = tf.constant(global_autoencoder_scifact_predictions_df.values, dtype=tf.float32)
scifact_embeddings_norm = tf.norm(scifact_embeddings_df_32, axis=1)

In [ ]:
evidence_column = []
for i, row in enumerate(global_autoencoder_scifact_predictions_tf):
  pred_topk = []
  successes = 0
  y_pred = tf.reshape(row, [1, -1])

  cos_sim_pred = tf.matmul(scifact_embeddings_df_32, y_pred, transpose_b=True) / tf.reshape((tf.norm(y_pred) * scifact_embeddings_norm), [-1, 1])
  top_k_sim_pred = tf.math.top_k(tf.reshape(cos_sim_pred, [-1]), k=scifact_topk).indices.numpy()

  for index in top_k_sim_pred:
    pred_topk.append(scifact_evidence_embeddings_corpus.iloc[index]['evidence'])
  evidence_column.append(pred_topk)

In [ ]:
scifact_topk_results = scifact_test.copy()
scifact_topk_results['topk'] = evidence_column

In [ ]:
os.makedirs('current-data-dump/nomic-scifact-autoencoder/nomic_autoencoder_predictions/')
scifact_topk_results.to_csv('current-data-dump/nomic-scifact-autoencoder/nomic_autoencoder_predictions/scifact_combined_topk_results.csv')

## Scifact Predict (Evidence + Pro + Qualitative)

In [ ]:
global_autoencoder_scifact_pro_predictions = global_scifact_pro_autoencoder_model.predict(scifact_test_embeddings)
global_autoencoder_scifact_pro_predictions_df = pd.DataFrame(global_autoencoder_scifact_pro_predictions)
global_autoencoder_scifact_pro_predictions_df.columns = [str(i) for i in global_autoencoder_scifact_pro_predictions_df.columns]

10/10 [==============================] - 0s 2ms/step


In [ ]:
scifact_pro_topk = 10
pred_pro_topk=[]

In [ ]:
# scifact_embeddings_df_32 = tf.cast(scifact_evidence_embeddings, dtype=tf.float32)
global_autoencoder_scifact_pro_predictions_tf = tf.constant(global_autoencoder_scifact_pro_predictions_df.values, dtype=tf.float32)
# scifact_embeddings_norm = tf.norm(scifact_embeddings_df_32, axis=1)

In [ ]:
evidence_pro_column = []
for i, row in enumerate(global_autoencoder_scifact_pro_predictions_tf):
  pred_pro_topk = []
  successes = 0
  y_pred = tf.reshape(row, [1, -1])

  cos_sim_pred = tf.matmul(scifact_embeddings_df_32, y_pred, transpose_b=True) / tf.reshape((tf.norm(y_pred) * scifact_embeddings_norm), [-1, 1])
  top_k_sim_pred = tf.math.top_k(tf.reshape(cos_sim_pred, [-1]), k=scifact_topk).indices.numpy()

  for index in top_k_sim_pred:
    pred_pro_topk.append(scifact_evidence_embeddings_corpus.iloc[index]['evidence'])
  evidence_pro_column.append(pred_pro_topk)

In [ ]:
scifact_pro_topk_results = scifact_test.copy()
scifact_pro_topk_results['topk'] = evidence_pro_column

In [ ]:
scifact_pro_topk_results.to_csv('current-data-dump/nomic-scifact-autoencoder/nomic_autoencoder_predictions/scifact_pro_topk_results.csv')

## Scifact Predict (Evidence + Counter + Qualitative)

In [ ]:
global_autoencoder_scifact_counter_predictions = global_scifact_counter_autoencoder_model.predict(scifact_test_embeddings)
global_autoencoder_scifact_counter_predictions_df = pd.DataFrame(global_autoencoder_scifact_counter_predictions)
global_autoencoder_scifact_counter_predictions_df.columns = [str(i) for i in global_autoencoder_scifact_counter_predictions_df.columns]

10/10 [==============================] - 0s 2ms/step


In [ ]:
scifact_counter_topk = 10
pred_counter_topk=[]

In [ ]:
# scifact_embeddings_df_32 = tf.cast(scifact_evidence_embeddings, dtype=tf.float32)
global_autoencoder_scifact_counter_predictions_tf = tf.constant(global_autoencoder_scifact_counter_predictions_df.values, dtype=tf.float32)
# scifact_embeddings_norm = tf.norm(scifact_embeddings_df_32, axis=1)

In [ ]:
evidence_counter_column = []
for i, row in enumerate(global_autoencoder_scifact_counter_predictions_tf):
  pred_counter_topk = []
  successes = 0
  y_pred = tf.reshape(row, [1, -1])

  cos_sim_pred = tf.matmul(scifact_embeddings_df_32, y_pred, transpose_b=True) / tf.reshape((tf.norm(y_pred) * scifact_embeddings_norm), [-1, 1])
  top_k_sim_pred = tf.math.top_k(tf.reshape(cos_sim_pred, [-1]), k=scifact_topk).indices.numpy()

  for index in top_k_sim_pred:
    pred_counter_topk.append(scifact_evidence_embeddings_corpus.iloc[index]['evidence'])
  evidence_counter_column.append(pred_counter_topk)

In [ ]:
scifact_counter_topk_results = scifact_test.copy()
scifact_counter_topk_results['topk'] = evidence_counter_column

In [ ]:
scifact_counter_topk_results.to_csv('current-data-dump/nomic-scifact-autoencoder/nomic_autoencoder_predictions/scifact_counter_topk_results.csv')

## Export Values

In [ ]:
result_df = {'gpr_success_rate': gpr_success_rate, 'eacl_topk_success_rate': eacl_topk_success_rate, 'eacl_topk': eacl_topk}
result_df = pd.DataFrame([result_df])
results_folder_path = 'current-data-dump/nomic-autoencoder/nomic_autoencoder_predictions/'
os.makedirs(results_folder_path, exist_ok=True)
results_file_path = f'{results_folder_path}novel_corpora_prediction.pkl'
with open(results_file_path, 'wb') as file:
  pickle.dump(result_df, file)
  print(f"File uploaded to {results_file_path}")

In [ ]:
evidence_result_df = {'persuade_topk_success_rate': persuade_topk_success_rate}
evidence_result_df = pd.DataFrame([evidence_result_df])
evidence_results_folder_path = 'current-data-dump/nomic-evidence-autoencoder/nomic_autoencoder_predictions/'
os.makedirs(evidence_results_folder_path, exist_ok=True)
evidence_results_file_path = f'{evidence_results_folder_path}persuade_corpus_prediction.pkl'
with open(evidence_results_file_path, 'wb') as file:
  pickle.dump(evidence_result_df, file)
  print(f"File uploaded to {evidence_results_file_path}")

In [ ]:
nomic_autoencoder_file_path = 'current-data-dump/nomic-autoencoder/nomic_autoencoder_predictions'
result = subprocess.run([f"osf -p sakjg upload -r --force {nomic_autoencoder_file_path}/ data-dump/nomic-autoencoder/nomic-autoencoder-predictions"], shell=True, capture_output=True, text=True)
print(result.stderr)
print(f"File: {nomic_autoencoder_file_path} uploaded at osfstorage")

In [ ]:
nomic_evidence_autoencoder_file_path = 'current-data-dump/nomic-evidence-autoencoder/nomic_autoencoder_predictions'
result = subprocess.run([f"osf -p sakjg upload -r --force {nomic_evidence_autoencoder_file_path}/ data-dump/nomic-evidence-autoencoder/nomic-autoencoder-predictions"], shell=True, capture_output=True, text=True)
print(result.stderr)
print(f"File: {nomic_evidence_autoencoder_file_path} uploaded at osfstorage")

In [ ]:
nomic_scifact_autoencoder_file_path = 'current-data-dump/nomic-scifact-autoencoder/nomic_autoencoder_predictions'
result = subprocess.run([f"osf -p sakjg upload -r --force {nomic_scifact_autoencoder_file_path}/ data-dump/nomic-scifact-autoencoder/nomic-autoencoder-predictions"], shell=True, capture_output=True, text=True)
print(result.stderr)
print(f"File: {nomic_scifact_autoencoder_file_path} uploaded at osfstorage")


File: current-data-dump/nomic-scifact-autoencoder/nomic_autoencoder_predictions uploaded at osfstorage
